In [1]:
import os
import time
import datetime
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader, TensorDataset, SequentialSampler, RandomSampler
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

from utils_data import  Vocabulary, Vectorizer, HeadQA, HeadQA_IR
from utils_data import parse_dataset, parse_ir_dataset, random_oversamplig, random_undersampling
from utils_data import filter_by_category, save_dataset_to_pickle, load_dataset_from_pickle
import training
from training import get_optimizer, train, train_ir, validate, validate_ir, evaluator, evaluator_ir, evaluate
from training import load_embeddings_from_file, make_embedding_matrix
from training import pad_seq, encoder_bert, encoder_bert_ir, encoder_bert_instance, encoder_bert_ir_instance
from training import evaluator_bert, evaluator_bert_ir

import transformers
from transformers.optimization import AdamW
from transformers import BertForSequenceClassification, BertConfig, BertTokenizer, BertModel, BertForMaskedLM


%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
BASE_BERT = 'dccuchile/bert-base-spanish-wwm-cased'

In [ ]:
from datasets import load_dataset

data_es = load_dataset('head_qa', 'es' )

In [ ]:
training, validation, testing = data_es['train'], data_es['validation'], data_es['test']

In [ ]:
# training_instances = parse_dataset(training)
# validation_instances = parse_dataset(validation)
# testing_instances = parse_dataset(testing)

# oversampled_training = random_oversamplig(training_instances)

In [ ]:
training_instances = load_dataset_from_pickle('../data/training_ir.pickle')
validation_instances = load_dataset_from_pickle('../data/validation_ir.pickle')
testing_instances = load_dataset_from_pickle('../data/testing_ir.pickle')
oversampled_training = load_dataset_from_pickle('../data/oversampled_training_ir.pickle')

In [ ]:
tokenizer = BertTokenizer.from_pretrained(BASE_BERT, do_lower_case=False)

In [ ]:
def pad_seq(x, seq_len=110, right_padding = False):
    z = np.zeros(seq_len, dtype=np.int32)
    n = min(seq_len, len(x))
    if right_padding:
        z[:n] = x[0:n]
    else:
        z[(seq_len - n):] = x[0:n]
    return z

In [ ]:
testing_instances[0]

In [ ]:
def encode_ir(samples, tokenizer):
    input_ids_0, input_ids_1, labels = [], [], []
    for item in samples:
        encoded_q = tokenizer.encode(item['question'], add_special_tokens = True)
        encoded_a = tokenizer.encode(item['answer'], add_special_tokens = True)
        padded_q = pad_seq(encoded_q, seq_len=30)
        padded_a = pad_seq(encoded_a, seq_len=30)
        input_ids_0.append(padded_q)
        input_ids_1.append(padded_a)
        labels.append(item['label'])
        
    attention_masks_0, attention_masks_1 = [], []
    for sent in input_ids_0:  
        att_mask = [int(token_id > 0) for token_id in sent]
        attention_masks_0.append(att_mask)
    
    for sent in input_ids_1:  
        att_mask = [int(token_id > 0) for token_id in sent]
        attention_masks_1.append(att_mask)
        
    return input_ids_0, attention_masks_0, input_ids_1, attention_masks_1, labels

In [ ]:
train_inputs_0, train_masks_0, train_inputs_1, train_masks_1, train_labels = encode_ir(oversampled_training, tokenizer)
valid_inputs_0, valid_masks_0, valid_inputs_1, valid_masks_1, valid_labels = encode_ir(validation_instances, tokenizer)
test_inputs_0, test_masks_0, test_inputs_1, test_masks_1, test_labels = encode_ir(testing_instances, tokenizer)

In [ ]:
train_inputs_0 = torch.tensor(train_inputs_0)
valid_inputs_0 = torch.tensor(valid_inputs_0)
test_inputs_0 = torch.tensor(test_inputs_0)

train_masks_0 = torch.tensor(train_masks_0)
valid_masks_0 = torch.tensor(valid_masks_0)
test_masks_0 = torch.tensor(test_masks_0)

train_inputs_1 = torch.tensor(train_inputs_1)
valid_inputs_1 = torch.tensor(valid_inputs_1)
test_inputs_1 = torch.tensor(test_inputs_1)

train_masks_1 = torch.tensor(train_masks_1)
valid_masks_1 = torch.tensor(valid_masks_1)
test_masks_1 = torch.tensor(test_masks_1)

train_labels = torch.tensor(train_labels)
valid_labels = torch.tensor(valid_labels)
test_labels = torch.tensor(test_labels)

In [ ]:
batch_size = 8

# Create the DataLoader for our training set.
train_data = TensorDataset(train_inputs_0, train_masks_0, train_inputs_1, train_masks_1, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set.
valid_data = TensorDataset(valid_inputs_0, valid_masks_0, valid_inputs_1, valid_masks_1, valid_labels)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=batch_size)

# Create the DataLoader for our test set.
test_data = TensorDataset(test_inputs_0, test_masks_0, test_inputs_1, test_masks_1, test_labels)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [ ]:
import time
import copy
import spacy
import pickle
import collections
from tqdm import tqdm_notebook, trange
from collections import Counter

In [ ]:
class BERT_QA(torch.nn.Module):
    """BERT model for classification.
    This module is composed of the BERT model with a linear layer on top of
    the pooled output.
    """
    def __init__(self, num_labels=1, pretrained_model='bert-base-uncased', seq_length=30):
        super(BERT_QA, self).__init__()
        self.num_labels = num_labels
        self.bert = BertModel.from_pretrained(pretrained_model)
        config = self.bert.config
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.cosine = nn.CosineSimilarity(dim=1)
        self.classifier = nn.Linear(config.hidden_size, 256)
        self.classifier_1 = nn.Linear(256, self.num_labels)
        nn.init.xavier_normal_(self.classifier.weight)

    def forward(self, input_ids_0, attention_mask_0, input_ids_1, attention_mask_1, labels=None, output_hidden_states=True):
        outputs_0 = self.bert(input_ids_0, attention_mask=attention_mask_0, output_hidden_states=output_hidden_states)
        outputs_1 = self.bert(input_ids_1, attention_mask=attention_mask_1, output_hidden_states=output_hidden_states)
        a = torch.cat(outputs_0[2][:4], dim=1)
        b = torch.cat(outputs_1[2][:4], dim=1)
        s = self.cosine(a, b)
        x = self.classifier(s)
        x = self.dropout(x)
        x = self.classifier_1(x)
        return F.softmax(x, dim=0)

    def freeze_bert_encoder(self):
        for param in self.bert.parameters():
            param.requires_grad = False

    def unfreeze_bert_encoder(self):
        for param in self.bert.parameters():
            param.requires_grad = True

In [ ]:
model = BERT_QA(pretrained_model=BASE_BERT)

In [ ]:
for batch in train_dataloader:    
    print(len(batch))
    b_input_ids_0, b_input_mask_0 = batch[0].long(), batch[1]
    b_input_ids_1, b_input_mask_1 = batch[2].long(), batch[3]
    b_labels = batch[4]
#     model.zero_grad()
    output = model(input_ids_0=b_input_ids_0, 
              attention_mask_0=b_input_mask_0, 
              input_ids_1=b_input_ids_1,
              attention_mask_1=b_input_mask_1,
              labels=b_labels)
    break

In [ ]:
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

In [ ]:
from transformers import get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

epochs = 4
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [ ]:
def flat_accuracy(preds, labels):
    pred_flat = np.round(preds, axis=0)
    labels_flat = labels
    print(pred_flat, labels_flat)
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
def train_model(model, train_dataloader, valid_dataloader, epochs):
    import random
    seed_val = 42
    random.seed(seed_val)
    np.random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)

    # Store the average loss after each epoch so we can plot them.
    loss_values = []
    epochs_results = []

    for epoch_i in range(0, epochs):
        print("")
        print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
        print('Training...')

        # Measure how long the training epoch takes.
        t0 = time.time()
        total_loss = 0
        model.train()
        for step, batch in enumerate(train_dataloader):
            if step % 40 == 0 and not step == 0:
                elapsed = format_time(time.time() - t0)
                print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
            b_input_ids_0, b_input_mask_0 = batch[0].long(), batch[1]
            b_input_ids_1, b_input_mask_1 = batch[2].long(), batch[3]
            b_labels = batch[4]
            model.zero_grad()
            out = model(input_ids_0=b_input_ids_0, 
                      attention_mask_0=b_input_mask_0, 
                      input_ids_1=b_input_ids_1,
                      attention_mask_1=b_input_mask_1,
                      labels=b_labels)
            b_labels = b_labels.unsqueeze(dim=1)
            loss = F.binary_cross_entropy(out, b_labels.float())
            total_loss += loss.item()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
        avg_train_loss = total_loss / len(train_dataloader)  
        loss_values.append(avg_train_loss)
        valid_acc, y_real, y_pred = valid_model(model, valid_dataloader)
        p, r, f1 = evaluate(y_real, y_pred)
        epochs_results.append([avg_train_loss, valid_acc, p, r, f1])

        print("")
        print("  Average training loss: {0:.2f}".format(avg_train_loss))
        print("  Training epoch took: {:}".format(format_time(time.time() - t0)))
    print("")
    print("Training complete!")
    return epochs_results

In [ ]:
def valid_model(model, validation_dataloader):
    print("Running Validation...")    
    y_true = []
    y_pred = []
    
    t0 = time.time()
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    
    for batch in validation_dataloader:
        b_input_ids_0, b_input_mask_0 = batch[0].long(), batch[1]
        b_input_ids_1, b_input_mask_1 = batch[2].long(), batch[3]
        with torch.no_grad(): 
            out = model(input_ids_0=b_input_ids_0, 
                      attention_mask_0=b_input_mask_0, 
                      input_ids_1=b_input_ids_1,
                      attention_mask_1=b_input_mask_1,
                      labels=b_labels)
        logits = out
        #logits = logits.detach().cpu().numpy()
        #label_ids = b_labels.to('cpu').numpy()
        tmp_eval_accuracy = flat_accuracy(np.array(logit), np.array(b_labels))
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1
        pred = torch.max(logits, dim=1)[1]
        y_true.append(b_labels)
        y_pred.append(pred)        
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))
    return eval_accuracy/nb_eval_steps, y_true, y_pred

In [ ]:
def test_model(model, test_dataloader):
    model.eval()
    predictions , true_labels = [], []

    for batch in test_dataloader:
        #batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        b_input_ids = b_input_ids.long()
        with torch.no_grad():
            outputs = model(b_input_ids, token_type_ids=None, 
                          attention_mask=b_input_mask)
        logits = outputs[0]
        # Move logits and labels to CPU
        #logits = logits.detach().cpu().numpy()
        #label_ids = b_labels.to('cpu').numpy()
        # Store predictions and true labels
        pred = torch.max(logits, dim=1)[1]
        predictions.append(pred)
        true_labels.append(b_labels)
    print('    DONE.')
    return true_labels, predictions

In [ ]:
epochs_results = train_model(model, train_dataloader, valid_dataloader, epochs=4)

In [ ]:
acc, points = evaluate(model, validation, encoder_bert, evaluator_bert)
acc, points

In [ ]:
acc, points = evaluate(model, testing, encoder_bert, evaluator_bert)
acc, points